In [ ]:
#used a lot from this notebook: https://www.kaggle.com/code/westoon/xgboost-catboost-1-0457/notebook
#used my own preprocessing

In [ ]:

import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils.fixes import parse_version


In [ ]:

import kagglehub

%env KAGGLE_USERNAME=lilyguizatoullina
%env KAGGLE_KEY=7bf2fe425f08db3506cb1ccfc3a0f223

!kaggle competitions download -c playground-series-s4e12


env: KAGGLE_USERNAME=lilyguizatoullina
env: KAGGLE_KEY=7bf2fe425f08db3506cb1ccfc3a0f223
 99% 75.0M/75.5M [00:02<00:00, 45.3MB/s]
100% 75.5M/75.5M [00:02<00:00, 32.8MB/s]


In [ ]:
!unzip playground-series-s4e12.zip

Archive:  playground-series-s4e12.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train = pd.read_csv('/content/train.csv')
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [ ]:
test = pd.read_csv('/content/test.csv')
test.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    800000 non-null  int64  
 1   Age                   787511 non-null  float64
 2   Gender                800000 non-null  object 
 3   Annual Income         770140 non-null  float64
 4   Marital Status        787664 non-null  object 
 5   Number of Dependents  726870 non-null  float64
 6   Education Level       800000 non-null  object 
 7   Occupation            560875 non-null  object 
 8   Health Score          750551 non-null  float64
 9   Location              800000 non-null  object 
 10  Policy Type           800000 non-null  object 
 11  Previous Claims       557198 non-null  float64
 12  Vehicle Age           799997 non-null  float64
 13  Credit Score          708549 non-null  float64
 14  Insurance Duration    799998 non-null  float64
 15  

In [ ]:
train['Policy Start Date'] = pd.DatetimeIndex(train['Policy Start Date'])
train['year'] = train['Policy Start Date'].dt.year
train['month'] = train['Policy Start Date'].dt.month
train['date'] = train['Policy Start Date'].dt.day
train['day'] = train['Policy Start Date'].dt.weekday
train.drop('Policy Start Date',axis=1,inplace=True)

train['Exercise Frequency'] = train['Exercise Frequency'].replace({'Rarely': 1, 'Monthly': 2, 'Weekly': 3, 'Daily': 4})

train['Customer Feedback'] = train['Customer Feedback'].replace({'Poor': 1, 'Average': 2, 'Good': 3})

train['Customer Feedback'] = train['Customer Feedback'].fillna(0)

<ipython-input-12-ae87eeb8f3dc>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Exercise Frequency'] = train['Exercise Frequency'].replace({'Rarely': 1, 'Monthly': 2, 'Weekly': 3, 'Daily': 4})
<ipython-input-12-ae87eeb8f3dc>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Customer Feedback'] = train['Customer Feedback'].replace({'Poor': 1, 'Average': 2, 'Good': 3})


In [ ]:
test['Policy Start Date'] = pd.DatetimeIndex(test['Policy Start Date'])
test['year'] = test['Policy Start Date'].dt.year
test['month'] = test['Policy Start Date'].dt.month
test['date'] = test['Policy Start Date'].dt.day
test['day'] = test['Policy Start Date'].dt.weekday
test.drop('Policy Start Date',axis=1,inplace=True)

test['Exercise Frequency'] = test['Exercise Frequency'].replace({'Rarely': 1, 'Monthly': 2, 'Weekly': 3, 'Daily': 4})

test['Customer Feedback'] = test['Customer Feedback'].replace({'Poor': 1, 'Average': 2, 'Good': 3})

test['Customer Feedback'] = test['Customer Feedback'].fillna(0)

<ipython-input-13-74c0a2bab3c8>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['Exercise Frequency'] = test['Exercise Frequency'].replace({'Rarely': 1, 'Monthly': 2, 'Weekly': 3, 'Daily': 4})
<ipython-input-13-74c0a2bab3c8>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['Customer Feedback'] = test['Customer Feedback'].replace({'Poor': 1, 'Average': 2, 'Good': 3})


In [ ]:
train['Number of Dependents'] = train['Number of Dependents'].fillna(0)
test['Number of Dependents'] = test['Number of Dependents'].fillna(0)

In [ ]:
train['Previous Claims'] = train['Previous Claims'].fillna(0)
test['Previous Claims'] = test['Previous Claims'].fillna(0)

In [ ]:
train['Marital Status'] = train['Marital Status'].fillna('Single')
test['Marital Status'] = test['Marital Status'].fillna('Single')

In [ ]:
train['Occupation'] = train['Occupation'].fillna('Unemployed')
test['Occupation'] = test['Occupation'].fillna('Unemployed')

In [ ]:
train["Age"] = train["Age"].fillna(train["Age"].mean())
test["Age"] = test["Age"].fillna(test["Age"].mean())

In [ ]:
#for column in ["Health Score", "Credit Score", "Annual Income"]:
#    train[column] = train[column].fillna(train[column].mode()[0])
#for column in ["Health Score", "Credit Score", "Annual Income"]:
#    test[column] = test[column].fillna(test[column].mode()[0])

In [ ]:
for column in ["Vehicle Age", "Insurance Duration"]:
    train[column] = train[column].fillna(-99)
for column in ["Vehicle Age", "Insurance Duration"]:
    test[column] = test[column].fillna(-99)

In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
#train['Annual Income'] = np.log1p(train['Annual Income'])
#test['Annual Income'] = np.log1p(test['Annual Income'])

In [ ]:
train.head()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,...,Location_Suburban,Location_Urban,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Smoking Status_No,Smoking Status_Yes,Property Type_Apartment,Property Type_Condo,Property Type_House
0,0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.0,5.0,1.0,...,False,True,False,False,True,True,False,False,False,True
1,1,39.0,31678.0,3.0,15.569731,1.0,12.0,694.0,2.0,2.0,...,False,False,False,True,False,False,True,False,False,True
2,2,23.0,25602.0,3.0,47.177549,1.0,14.0,NaN,3.0,3.0,...,True,False,False,False,True,False,True,False,False,True
3,3,21.0,141855.0,2.0,10.938144,1.0,0.0,367.0,1.0,1.0,...,False,False,True,False,False,False,True,True,False,False
4,4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.0,4.0,1.0,...,False,False,False,False,True,False,True,False,False,True


In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
matrix = round(train.select_dtypes(include=numerics).corr(),2)
print(matrix)

                       id  Age  Annual Income  Number of Dependents  \
id                    1.0 -0.0          -0.00                   0.0   
Age                  -0.0  1.0          -0.00                   0.0   
Annual Income        -0.0 -0.0           1.00                  -0.0   
Number of Dependents  0.0  0.0          -0.00                   1.0   
Health Score          0.0  0.0           0.03                   0.0   
Previous Claims      -0.0  0.0           0.04                  -0.0   
Vehicle Age          -0.0 -0.0          -0.00                   0.0   
Credit Score          0.0  0.0          -0.20                   0.0   
Insurance Duration   -0.0 -0.0           0.00                  -0.0   
Customer Feedback     0.0  0.0          -0.03                   0.0   
Exercise Frequency    0.0  0.0          -0.00                  -0.0   
Premium Amount       -0.0 -0.0          -0.01                  -0.0   
year                 -0.0 -0.0          -0.01                   0.0   
month 

In [ ]:
train.isnull().sum()

,0
id,0
Age,0
Annual Income,44949
Number of Dependents,0
Health Score,74076
Previous Claims,0
Vehicle Age,0
Credit Score,137882
Insurance Duration,0
Customer Feedback,0


In [ ]:
from sklearn.model_selection import train_test_split
X, y = train.drop('Premium Amount', axis=1), train[['Premium Amount']]

In [ ]:
#meow

In [ ]:
pip install catboost

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
target = 'Premium Amount'
numeric_cols = train.select_dtypes(include=[np.number]).columns.drop(target, errors='ignore').tolist()
cat_cols = train.select_dtypes(exclude=[np.number, 'datetime64[ns]']).columns.tolist()

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(columns=target)
y = train[target]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Identify categorical feature indices for CatBoost
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols if c in X_train.columns]

In [ ]:
params = {
    'iterations': 1100,
    'learning_rate': 0.05,
    'depth': 8,
    'loss_function': 'RMSE',
    'random_seed': 42,
    'task_type': 'GPU',
    'verbose': 0
}

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_scores = []
models = []

for i, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    model = CatBoostRegressor(**params, cat_features=cat_features_indices)

    model.fit(X_train_fold, np.log1p(y_train_fold),
              eval_set=(X_val_fold, np.log1p(y_val_fold)),
              early_stopping_rounds=50,
              use_best_model=True)

    y_pred_fold = np.expm1(model.predict(X_val_fold))
    fold_rmsle = np.sqrt(mean_squared_log_error(y_val_fold, y_pred_fold))
    fold_scores.append(fold_rmsle)
    models.append(model)
    print(f"Fold {i+1} RMSLE: {fold_rmsle}")

mean_rmsle = np.mean(fold_scores)
print(f"Mean RMSLE: {mean_rmsle:.5f}")

Fold 1 RMSLE: 1.0495922629083314
Fold 2 RMSLE: 1.0483532887926899
Fold 3 RMSLE: 1.048441990325832
Fold 4 RMSLE: 1.0499752975227643
Fold 5 RMSLE: 1.0479190117391333
Mean RMSLE: 1.04886


In [ ]:
test_preds_list = []
for i, model in enumerate(models):
    preds = model.predict(test)
    test_preds_list.append(np.expm1(preds))

test_preds_array = np.column_stack(test_preds_list)
test_final_preds = np.mean(test_preds_array, axis=1)

In [ ]:
output= pd.DataFrame(test['id'])
cat_output = test_final_preds
output['Premium Amount']= cat_output
output.to_csv("submission_cat.csv", index = None)

In [ ]:
# One hot encoding cat cols for XGB
for idx, dataset in enumerate([train, test]):
    i_dummy = pd.get_dummies(dataset[cat_cols], drop_first=True).astype(int)

    dataset.drop(columns=[
        'Gender', 'Marital Status', 'Education Level', 'Occupation',
        'Location', 'Policy Type', 'Customer Feedback', 'Smoking Status',
        'Exercise Frequency', 'Property Type', 'month', 'day'
    ], inplace=True)

    dataset = pd.concat([dataset, i_dummy], axis=1)

    if idx == 0:
        train = dataset
    else:
        test = dataset

In [ ]:
X = train.drop(columns=target)
y = train[target]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

params = {'n_estimators': 1100,
          'colsample_bytree': 1.0,
          'learning_rate': 0.02,
          'max_depth': 10,
          'subsample': 0.8,
          'random_state':42,
          'eval_metric':'rmsle',
          'tree_method':'hist',
          'device':'gpu',
          'early_stopping_rounds':50,
          'verbose':200}

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_scores = []
models = []

for i, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Initialize XGB
    model = XGBRegressor(**params)

    model.fit(X_train_fold, np.log1p(y_train_fold), eval_set=[(X_val_fold, np.log1p(y_val_fold))], verbose=200)


    y_pred_fold = np.expm1(model.predict(X_val_fold))

    fold_rmsle = np.sqrt(mean_squared_log_error(y_val_fold, y_pred_fold))
    fold_scores.append(fold_rmsle)
    models.append(model)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:41:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmsle:0.16545
[200]	validation_0-rmsle:0.15862
[308]	validation_0-rmsle:0.15862


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:41:27] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmsle:0.16499


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:41:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


...

[200]	validation_0-rmsle:0.15832
[259]	validation_0-rmsle:0.15832
[0]	validation_0-rmsle:0.16537


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:41:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[200]	validation_0-rmsle:0.15872
[285]	validation_0-rmsle:0.15872
[0]	validation_0-rmsle:0.16487


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:42:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[200]	validation_0-rmsle:0.15822
[287]	validation_0-rmsle:0.15822


In [ ]:
val_preds_list = []
for i, model in enumerate(models):
    preds = model.predict(X_val)
    val_preds_list.append(np.expm1(preds))

val_preds_array = np.column_stack(val_preds_list)
val_final_preds = np.mean(val_preds_array, axis=1)

print("AVG Val Pred RMSLE:", np.sqrt(mean_squared_log_error(y_val, val_final_preds)))

AVG Val Pred RMSLE: 1.0475715777141617


In [ ]:
test_preds_list = []
for i, model in enumerate(models):
    preds = model.predict(test)
    test_preds_list.append(np.expm1(preds))

test_preds_array = np.column_stack(test_preds_list)
test_final_preds = np.mean(test_preds_array, axis=1)

In [ ]:
output= pd.DataFrame(test['id'])
xgb_output = test_final_preds
output['Premium Amount']= xgb_output
output.to_csv("submission_xgb.csv", index = None)

In [ ]:
submission_cat = pd.read_csv("submission_cat.csv")
submission_xgb = pd.read_csv("submission_xgb.csv")


submission_avg = submission_cat.copy()
submission_avg['Premium Amount'] = (
    0.6 * submission_cat['Premium Amount'] +
    0.4 * submission_xgb['Premium Amount']
)


submission_avg.to_csv("submission.csv", index=False)

In [ ]:
!kaggle competitions submit playground-series-s4e12  -f submission.csv -m "submission"

100% 19.8M/19.8M [00:01<00:00, 12.9MB/s]
Successfully submitted to Regression with an Insurance Dataset